The methods used to generate the filter terms involved looking at the most frequent words/client names for clues about what types of establishments were mentioned in this data set. 

Developing the filters took a fair amount of "human" sleuthing while looking at TF-IDF scores, frequency counts of Client names, as well as just general knowledge of common Spanish words used to refer to certain establishment types. Especially with all the noise provided by the Clients referred to only by a proper name, the filtered data is a proportionally small figure- but, significant.

In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

# Load in the Client Name data
# Make sure all names uppercase (there are some mixed instances)
pd.set_option('display.max_rows', 50)
vf = pd.read_csv('/data/arpit.goel/18_InventoryPrediction/02.ExtractedData/cliente_tabla.csv',header=0)
vf['NombreCliente'] = vf['NombreCliente'].str.upper()

In [9]:
# Begin with a scan of the Client Name Data based on Top Frequency Client Names
# Notice there are a lot of Proper Names
vf['NombreCliente'].value_counts()[0:200]

NO IDENTIFICADO    281670
LUPITA               4863
MARY                 3016
LA PASADITA          2426
LA VENTANITA         2267
LA GUADALUPANA       1299
ROSY                 1246
ALEX                 1242
GABY                 1238
LA ESCONDIDA         1216
PATY                 1145
LA ESPERANZA         1139
HERNANDEZ            1129
LA CHIQUITA          1118
DANY                 1082
GARCIA               1062
JUQUILITA            1022
MARTINEZ              987
LA TIENDITA           914
3 HERMANOS            913
DIANA                 894
MI TIENDITA           881
SAN JOSE              851
LA PROVIDENCIA        842
GONZALEZ              826
                    ...  
VERONICA              256
CECY                  253
LAS GEMELAS           252
LA BODEGUITA          250
MARIBEL               249
RUIZ                  249
EL CRUCERO            249
LUZ                   248
EL SOL                248
LA POBLANITA          248
CHAYITO               247
MONICA                244
ALMA        

In [10]:
# Let's also generate a list of individual word frequency across all names
def tfidf_score_list(vf2, list_len):
    from sklearn.feature_extraction.text import TfidfVectorizer
    v = TfidfVectorizer()

    vf2['New'] = 'na'
    a = " ".join(vf2['NombreCliente'])
    vf2['New'][0] = a

    tfidf = v.fit_transform(vf2['New'])

    feature_names = v.get_feature_names()

    freq = []
    doc = 0
    feature_index = tfidf[doc,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf[doc, x] for x in feature_index])
    for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
            freq.append((w.encode('utf-8'),s))
    
    del vf2['New']
    
    import numpy as np
    names = ['word','score']
    formats = ['S50','f8']
    dtype = dict(names = names, formats=formats)
    array = np.array(freq, dtype=dtype)

    b = np.sort(array, order='score')
    
    if list_len > len(b)+1:
        list_len = len(b)+1
    for i in range(1,list_len):
        print(b[-i])

In [11]:
tfidf_score_list(vf, 200)

/data/arpit.goel/anacondainstall/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


('no', 0.6888126004393861)
('identificado', 0.6849292193081505)
('la', 0.14990532034895288)
('el', 0.08328478631485127)
('abarrotes', 0.0800093233318993)
('de', 0.060769929775020375)
('maria', 0.046819424108208656)
('miscelanea', 0.038050812737366206)
('super', 0.03520332413080589)
('los', 0.02841652592336794)
('san', 0.025741686070920162)
('oxxo', 0.02257321668206611)
('del', 0.021668634477420134)
('garcia', 0.021376833766244014)
('hernandez', 0.02099506116912192)
('jose', 0.019835153342196838)
('lupita', 0.019261278610217134)
('gonzalez', 0.01775850494766011)
('martinez', 0.01764664800837593)
('lopez', 0.017150586799376526)
('mini', 0.015331695699712036)
('rodriguez', 0.014891562960354718)
('las', 0.01388971385198337)
('don', 0.012542567235386944)
('comodin', 0.01238450851683321)
('guadalupe', 0.012357760118308733)
('mary', 0.012270219904955897)
('jesus', 0.012024620973049328)
('ramirez', 0.011995440901931718)
('juan', 0.011966260830814104)
('casa', 0.011929785741917089)
('ag', 0.011

Again, notice the prevalence of personal names. By looking at a long enough list, however, we can start to see some other useful terms appear such as particles of speech (i.e. 'ag', 'los', 'san') or more useful words like "super", "oxxo", "mini", "comodin". If I found a word that I thought represent a type of establishment, I'd double check it by doing a single search. If that search was fruitful and had a good amount of relevant results, I'd add it to my filter. An example of a single term search is below:

In [12]:
print(vf[vf['NombreCliente'].str.contains('.*CAFE.*')])

        Cliente_ID                         NombreCliente
78            1438                 CAFETRIA PREPARATORIA
1095          5045         CAFETERIA DE LA SECUNDARIA 13
1098          5048                     CAFETERIA PREPA 2
1233          5416                             CAFETERIA
1318          5612        CAFETERIA NORMAL DE PROFESORES
1446          5908      CAFETERIA FACULTAD DE PSICOLOGIA
2005          7463                  CAFETERIA DE LA UAEM
2150          7922                       ICATI CAFETERIA
2306          8317         CAFETERIA FACULTAD DE DERECHO
2308          8319  CAFETERIA FACULTAD DE ADMINISTRACION
2573          8979                             CAFETERIA
2864          9956                               LA CAFE
3147         10753                  CAFETERIA LA CARRETA
3207         10844                  CAFETERIA EL JUBILEO
3209         10846                  CAFETERIA LA CENTRAL
3240         10913                  CAFETERIA DON ARTURO
3243         10918             

The result is a filter derived from hand-picking the best, most common, most interesting terms I could determine.

In [13]:
# --- Begin Filtering for specific terms

# Note that the order of filtering is significant.
# For example: 
# The regex of .*ERIA.* will assign "FRUITERIA" to 'Eatery' rather than 'Fresh Market'.
# In other words, the first filters to occur have a bigger priority.

def filter_specific(vf2):
    
    # Known Large Company / Special Group Types
    vf2['NombreCliente'] = vf2['NombreCliente'].str.replace('.*REMISION.*','Consignment')
    vf2['NombreCliente'] = vf2['NombreCliente'].replace(['.*WAL MART.*','.*SAMS CLUB.*'],'Walmart', regex=True)
    vf2['NombreCliente'] = vf2['NombreCliente'].str.replace('.*OXXO.*','Oxxo Store')
    vf2['NombreCliente'] = vf2['NombreCliente'].str.replace('.*CONASUPO.*','Govt Store')
    vf2['NombreCliente'] = vf2['NombreCliente'].str.replace('.*BIMBO.*','Bimbo Store')

    

    # General term search for a random assortment of words I picked from looking at
    # their frequency of appearance in the data and common spanish words for these categories
    vf2['NombreCliente'] = vf2['NombreCliente'].replace(['.*COLEG.*','.*UNIV.*','.*ESCU.*','.*INSTI.*',\
                                                        '.*PREPAR.*'],'School', regex=True)
    vf2['NombreCliente'] = vf2['NombreCliente'].str.replace('.*PUESTO.*','Post')
    vf2['NombreCliente'] = vf2['NombreCliente'].replace(['.*FARMA.*','.*HOSPITAL.*','.*CLINI.*'],'Hospital/Pharmacy', regex=True)
    vf2['NombreCliente'] = vf2['NombreCliente'].replace(['.*CAFE.*','.*CREMERIA.*','.*DULCERIA.*',\
                                                        '.*REST.*','.*BURGER.*','.*TACO.*', '.*TORTA.*',\
                                                        '.*TAQUER.*','.*HOT DOG.*',\
                                                        '.*COMEDOR.*', '.*ERIA.*','.*BURGU.*'],'Eatery', regex=True)
    vf2['NombreCliente'] = vf2['NombreCliente'].str.replace('.*SUPER.*','Supermarket')
    vf2['NombreCliente'] = vf2['NombreCliente'].replace(['.*COMERCIAL.*','.*BODEGA.*','.*DEPOSITO.*',\
                                                            '.*ABARROTES.*','.*MERCADO.*','.*CAMBIO.*',\
                                                        '.*MARKET.*','.*MART .*','.*MINI .*',\
                                                        '.*PLAZA.*','.*MISC.*','.*ELEVEN.*','.*EXP.*',\
                                                         '.*SNACK.*', '.*PAPELERIA.*', '.*CARNICERIA.*',\
                                                         '.*LOCAL.*','.*COMODIN.*','.*PROVIDENCIA.*'
                                                        ],'General Market/Mart'\
                                                       , regex=True)

    vf2['NombreCliente'] = vf2['NombreCliente'].replace(['.*VERDU.*','.*FRUT.*'],'Fresh Market', regex=True)
    vf2['NombreCliente'] = vf2['NombreCliente'].replace(['.*HOTEL.*','.*MOTEL.*'],'Hotel', regex=True)

In [14]:
filter_specific(vf)

In [15]:
# --- Begin filtering for more general terms
# The idea here is to look for names with particles of speech that would
# not appear in a person's name.
# i.e. "Individuals" should not contain any participles or numbers in their names.
def filter_participle(vf2):
    vf2['NombreCliente'] = vf2['NombreCliente'].replace([
            '.*LA .*','.*EL .*','.*DE .*','.*LOS .*','.*DEL .*','.*Y .*', '.*SAN .*', '.*SANTA .*',\
            '.*AG .*','.*LAS .*','.*MI .*','.*MA .*', '.*II.*', '.*[0-9]+.*'\
    ],'Small Franchise', regex=True)

In [16]:
filter_participle(vf)

In [17]:
# Any remaining entries should be "Individual" Named Clients, there are some outliers.
# More specific filters could be used in order to reduce the percentage of outliers in this final set.
def filter_remaining(vf2):
    def function_word(data):
        # Avoid the single-words created so far by checking for upper-case
        if (data.isupper()) and (data != "NO IDENTIFICADO"): 
            return 'Individual'
        else:
            return data
    vf2['NombreCliente'] = vf2['NombreCliente'].map(function_word)

In [18]:
filter_remaining(vf)

With the filtering complete, let's look at the breakdown of how the data is classified now:

In [19]:
vf['NombreCliente'].value_counts()

Individual             353145
NO IDENTIFICADO        281670
Small Franchise        160501
General Market/Mart     66416
Eatery                  30419
Supermarket             16019
Oxxo Store               9313
Hospital/Pharmacy        5798
School                   5705
Post                     2667
Hotel                    1127
Fresh Market             1069
Govt Store                959
Bimbo Store               320
Walmart                   220
Consignment                14
Name: NombreCliente, dtype: int64

Finally, we can apply these new tags on the actual Training and Test data sets that have been provided!

In [24]:
trdf = pd.read_csv('/data/arpit.goel/18_InventoryPrediction/02.ExtractedData/train.csv',header=0)
trdf_stores = trdf.merge(vf.drop_duplicates(subset=['Cliente_ID']), how="left")

In [25]:
tsdf = pd.read_csv('/data/arpit.goel/18_InventoryPrediction/02.ExtractedData/test.csv',header=0)
tsdf_stores = tsdf.merge(vf.drop_duplicates(subset=['Cliente_ID']), how="left")

Write the data to file to save it for a new session....

In [26]:
trdf.to_csv('/data/arpit.goel/18_InventoryPrediction/04.FeatureEngineering/train_with_cnames.csv')
tsdf.to_csv('/data/arpit.goel/18_InventoryPrediction/04.FeatureEngineering/test_with_cnames.csv')